In [ ]:
import cv2
import time
import threading
import smtplib
import os
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from ultralytics import YOLO
import winsound

# 🧠 Load mô hình YOLOv8
model = YOLO('../fire_yolov8_tested.pt')

# ⚠️ Thời gian giữa các lần cảnh báo
last_alert_time = 0
alert_interval = 5  # giây

# ✅ Thông tin email
SENDER_EMAIL = "baozzskt2k4@gmail.com"
RECEIVER_EMAIL = "naythipha2k4@gmail.com"
APP_PASSWORD = "xcqo cjlw meyi ocrk"  # Mật khẩu ứng dụng Gmail

def play_alert_sound():
    winsound.Beep(1000, 500)  # Tần số 1000Hz trong 0.5s


def send_email_alert(image_path):
    subject = "🔥 CẢNH BÁO CHÁY"
    body = "Hệ thống đã phát hiện cháy! Xem ảnh kèm theo để xác minh khu vực bị ảnh hưởng."

    message = MIMEMultipart()
    message["From"] = SENDER_EMAIL
    message["To"] = RECEIVER_EMAIL
    message["Subject"] = subject
    message.attach(MIMEText(body, "plain"))

    try:
        # Gắn ảnh cháy vào email
        with open(image_path, 'rb') as img_file:
            img_data = img_file.read()
            image = MIMEImage(img_data, name=os.path.basename(image_path))
            message.attach(image)

        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(SENDER_EMAIL, APP_PASSWORD)
        server.send_message(message)
        server.quit()
        print("✅ Email cảnh báo đã được gửi")
    except Exception as e:
        print("❌ Lỗi khi gửi email:", e)


# 📷 Mở webcam
cap = cv2.VideoCapture(0)

# 🔄 Vòng lặp
while True:
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    # 🔍 Dự đoán cháy
    results = model.predict(source=frame, save=False, imgsz=416, conf=0.5, device='cpu')
    boxes = results[0].boxes
    annotated_frame = results[0].plot()

    # 🧠 Nếu có cháy
    if len(boxes) > 0:
        current_time = time.time()
        if current_time - last_alert_time > alert_interval:
            print("🔥 CẢNH BÁO: PHÁT HIỆN CHÁY!")
            # Lưu ảnh tạm thời
            image_path = "fire_detected.jpg"
            cv2.imwrite(image_path, annotated_frame)

            # Cảnh báo âm thanh + gửi email
            threading.Thread(target=play_alert_sound).start()
            threading.Thread(target=send_email_alert, args=(image_path,)).start()

            last_alert_time = current_time

    # ⏱️ Tính FPS
    fps = 1.0 / (time.time() - start_time)
    cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # 🪟 Hiển thị
    cv2.imshow("🔥 Fire Detection", annotated_frame)

    # ⏹️ Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 🧹 Dọn dẹp
cap.release()
cv2.destroyAllWindows()



0: 320x416 (no detections), 31.6ms
Speed: 1.2ms preprocess, 31.6ms inference, 0.2ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 30.8ms
Speed: 0.8ms preprocess, 30.8ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 29.2ms
Speed: 0.9ms preprocess, 29.2ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 26.5ms
Speed: 0.8ms preprocess, 26.5ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 28.4ms
Speed: 0.8ms preprocess, 28.4ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 30.6ms
Speed: 0.7ms preprocess, 30.6ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 27.8ms
Speed: 0.8ms preprocess, 27.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 30.6ms
Speed: 0.8ms preprocess, 30.6ms i

Gửi cảnh báo về Telegram

In [1]:
import cv2
import time
import threading
import requests
from ultralytics import YOLO
import winsound

# 🧠 Load mô hình YOLOv8
model = YOLO('../fire_yolov8_tested.pt')

# ⚠️ Thời gian giữa các lần cảnh báo
last_alert_time = 0
alert_interval = 5  # giây

# ✅ Cấu hình Telegram
TELEGRAM_BOT_TOKEN = '7571823960:AAHM4mioG7DXv96P5JtUDIo8kE23WWDEYCE'
CHAT_ID = '5880384906'  # Lấy từ getUpdates

def play_alert_sound():
    winsound.Beep(1000, 500)

def send_telegram_alert(image_path):
    message = "🔥 CẢNH BÁO: Phát hiện cháy tại khu vực giám sát!"
    
    # Gửi tin nhắn
    requests.post(
        f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage",
        data={"chat_id": CHAT_ID, "text": message}
    )

    # Gửi ảnh
    with open(image_path, 'rb') as photo:
        requests.post(
            f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendPhoto",
            files={"photo": photo},
            data={"chat_id": CHAT_ID}
        )

# 📷 Mở webcam
cap = cv2.VideoCapture(0)

# 🔄 Vòng lặp
while True:
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        break

    # 🔍 Dự đoán
    results = model.predict(source=frame, save=False, imgsz=416, conf=0.5, device='cpu')
    boxes = results[0].boxes
    annotated_frame = results[0].plot()

    # 🧠 Nếu phát hiện cháy
    if len(boxes) > 0:
        current_time = time.time()
        if current_time - last_alert_time > alert_interval:
            print("🔥 CẢNH BÁO: PHÁT HIỆN CHÁY!")
            image_path = "fire_detected.jpg"
            cv2.imwrite(image_path, annotated_frame)

            threading.Thread(target=play_alert_sound).start()
            threading.Thread(target=send_telegram_alert, args=(image_path,)).start()

            last_alert_time = current_time

    # ⏱️ Tính FPS
    fps = 1.0 / (time.time() - start_time)
    cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # 🪟 Hiển thị
    cv2.imshow("🔥 Fire Detection", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 🧹 Dọn dẹp
cap.release()
cv2.destroyAllWindows()



0: 320x416 (no detections), 38.7ms
Speed: 2.1ms preprocess, 38.7ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 31.5ms
Speed: 1.1ms preprocess, 31.5ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 30.9ms
Speed: 0.8ms preprocess, 30.9ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 31.1ms
Speed: 1.0ms preprocess, 31.1ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 30.1ms
Speed: 0.8ms preprocess, 30.1ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 32.2ms
Speed: 0.8ms preprocess, 32.2ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 26.8ms
Speed: 0.8ms preprocess, 26.8ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 29.9ms
Speed: 0.9ms preprocess, 29.9ms i

Ứng dụng bằng TKinter

In [5]:
import cv2
import time
import threading
import requests
import winsound
from tkinter import *
from PIL import Image, ImageTk
from ultralytics import YOLO

# 🧠 Load mô hình YOLOv8
model = YOLO('../fire_yolov8_tested.pt')

# ⚠️ Cảnh báo và Telegram
last_alert_time = 0
alert_interval = 5
TELEGRAM_BOT_TOKEN = '7571823960:AAHM4mioG7DXv96P5JtUDIo8kE23WWDEYCE'
CHAT_ID = '5880384906'

# 🎵 Phát âm thanh cảnh báo
def play_alert_sound():
    winsound.Beep(1000, 500)

# 📤 Gửi cảnh báo Telegram
def send_telegram_alert(image_path):
    message = "🔥 CẢNH BÁO: Phát hiện cháy tại khu vực giám sát!"
    requests.post(
        f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage",
        data={"chat_id": CHAT_ID, "text": message}
    )
    with open(image_path, 'rb') as photo:
        requests.post(
            f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendPhoto",
            files={"photo": photo},
            data={"chat_id": CHAT_ID}
        )

# 🧠 Biến kiểm soát
running = False
cap = None

# 🔁 Vòng lặp xử lý video
def video_loop():
    global last_alert_time
    if not running:
        return

    ret, frame = cap.read()
    if not ret:
        return

    start_time = time.time()
    results = model.predict(source=frame, save=False, imgsz=416, conf=0.5, device='cpu')
    boxes = results[0].boxes
    annotated_frame = results[0].plot()

    # 🎯 Tính FPS
    fps = 1.0 / (time.time() - start_time)
    cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    # 🕒 Hiển thị ngày giờ hiện tại
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    cv2.putText(annotated_frame, f"Time: {current_time}", (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    if len(boxes) > 0 and time.time() - last_alert_time > alert_interval:
        print("🔥 CẢNH BÁO: PHÁT HIỆN CHÁY!")
        image_path = "fire_detected.jpg"
        cv2.imwrite(image_path, annotated_frame)
        threading.Thread(target=play_alert_sound).start()
        threading.Thread(target=send_telegram_alert, args=(image_path,)).start()
        last_alert_time = time.time()

    # 🖼️ Hiển thị ảnh lên giao diện
    frame_rgb = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(frame_rgb)
    imgtk = ImageTk.PhotoImage(image=img)
    panel.imgtk = imgtk
    panel.config(image=imgtk)

    root.after(10, video_loop)

# ▶️ Bắt đầu
def start_monitoring():
    global cap, running
    if not running:
        cap = cv2.VideoCapture(0)
        running = True
        video_loop()

# ⏹️ Dừng lại
def stop_monitoring():
    global running, cap
    running = False
    if cap:
        cap.release()

# 🪟 Giao diện Tkinter
root = Tk()
root.title("🔥 Ứng dụng Giám sát Cháy")

panel = Label(root)
panel.pack()

btn_start = Button(root, text="▶️ Bắt đầu Giám sát", command=start_monitoring, bg="green", fg="white", font=("Arial", 12))
btn_start.pack(fill="x", padx=10, pady=5)

btn_stop = Button(root, text="⏹️ Dừng lại", command=stop_monitoring, bg="red", fg="white", font=("Arial", 12))
btn_stop.pack(fill="x", padx=10, pady=5)

root.protocol("WM_DELETE_WINDOW", lambda: (stop_monitoring(), root.destroy()))
root.mainloop()



0: 320x416 (no detections), 38.0ms
Speed: 1.5ms preprocess, 38.0ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 45.9ms
Speed: 1.4ms preprocess, 45.9ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 37.3ms
Speed: 1.1ms preprocess, 37.3ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 30.3ms
Speed: 1.0ms preprocess, 30.3ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 27.1ms
Speed: 0.9ms preprocess, 27.1ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 29.3ms
Speed: 0.8ms preprocess, 29.3ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 30.3ms
Speed: 1.1ms preprocess, 30.3ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 416)

0: 320x416 (no detections), 27.7ms
Speed: 0.9ms preprocess, 27.7ms i